<a href="https://colab.research.google.com/github/aromaldev/OCR-Malayalam/blob/master/Preprocessing/Noise%20Removal%20%26%20Binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Modules
!pip install bm3d
!pip install opencv-rolling-ball

     |████████████████████████████████| 8.4MB 3.9MB/s 
  Created wheel for opencv-rolling-ball: filename=opencv_rolling_ball-1.0.1-cp37-none-any.whl size=6896 sha256=b49e93be721a5a8343097dd04ef98e219410c4efcdb73cf83cd76f299e603ed1
  Stored in directory: /root/.cache/pip/wheels/2c/e6/f1/98b88182253ed128cd894b374378bdbd44211601329d78f7c7
Successfully built opencv-rolling-ball


In [ ]:
#@title Utils
import numpy as np
from skimage.util.shape import view_as_windows
from PIL import Image
from matplotlib import pyplot as plt
def threshold_feng(img, w_size1=15, w_size2=30,k1=0.15, k2=0.01, alpha1=0.1) :
        # Obtaining rows and cols
        rows, cols = img.shape
        i_rows, i_cols = rows + 1, cols + 1

        # Computing integral images
        # Leaving first row and column in zero for convenience
        integ = np.zeros((i_rows, i_cols), np.float)
        sqr_integral = np.zeros((i_rows, i_cols), np.float)

        integ[1:, 1:] = np.cumsum(np.cumsum(img.astype(np.float), axis=0), axis=1)
        sqr_img = np.square(img.astype(np.float))
        sqr_integral[1:, 1:] = np.cumsum(np.cumsum(sqr_img, axis=0), axis=1)

        # Defining grid
        x, y = np.meshgrid(np.arange(1, i_cols), np.arange(1, i_rows))

        # Obtaining local coordinates
        hw_size = w_size1 // 2
        x1 = (x - hw_size).clip(1, cols)
        x2 = (x + hw_size).clip(1, cols)
        y1 = (y - hw_size).clip(1, rows)
        y2 = (y + hw_size).clip(1, rows)

        # Obtaining local areas size
        l_size = (y2 - y1 + 1) * (x2 - x1 + 1)

        # Computing sums
        sums = (integ[y2, x2] - integ[y2, x1 - 1] -
                integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
        sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

        # Computing local means
        means = sums / l_size

        # Computing local standard deviation
        stds = np.sqrt(sqr_sums / l_size - np.square(means))

        # Obtaining windows
        padded_img = np.ones((rows + w_size1 - 1, cols + w_size1 - 1)) * np.nan
        padded_img[hw_size: -hw_size, hw_size: -hw_size] = img

        winds = view_as_windows(padded_img, (w_size1, w_size1))

        # Obtaining maximums and minimums
        mins = np.nanmin(winds, axis=(2, 3))

        # Obtaining local coordinates for std range calculations
        hw_size = w_size2 // 2
        x1 = (x - hw_size).clip(1, cols)
        x2 = (x + hw_size).clip(1, cols)
        y1 = (y - hw_size).clip(1, rows)
        y2 = (y + hw_size).clip(1, rows)

        # Obtaining local areas size
        l_size = (y2 - y1 + 2) * (x2 - x1 + 2)

        # Computing sums
        sums = (integ[y2, x2] - integ[y2, x1 - 1] -
                integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
        sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

        # Computing local means2
        means2 = sums / l_size

        # Computing standard deviation range
        std_ranges = np.sqrt(sqr_sums / l_size - np.square(means2))

        # Computing normalized standard deviations and extra alpha parameters
        n_stds = stds / std_ranges
        n_sqr_std = np.square(n_stds)
        alpha2 = k1 * n_sqr_std
        alpha3 = k2 * n_sqr_std

        threshold = ((1 - alpha1) * means + alpha2 * n_stds * (means - mins) + alpha3 * mins)

        return threshold

def save(path, img):
        """
        Save the image in the path
        """
        tmp = np.asarray(img*255.0, dtype=np.uint8)
        Image.fromarray(tmp).save(path)

def plot_images(*args,titles = [],view=1,figsize = (30,50)):
        if(len(args)>1):
                if(view==0):
                        fig, ax = plt.subplots(len(args), 1, figsize=figsize)
                else:
                        fig, ax = plt.subplots(1,len(args),figsize=figsize) 
                for index,image in enumerate(args):
                        ax[index].imshow(image,'gray')
                        ax[index].axis('off')
                        if(titles):
                                ax[index].set_title(titles[index])
        else:
                plt.imshow(args[0],'gray')
                
        plt.show()

In [ ]:
#@title Preprocessing Module
import numpy as np
from skimage import io, img_as_float, color
from scipy import signal
from skimage.filters import threshold_otsu, threshold_sauvola, gaussian
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
import cv2
from skimage.morphology import remove_small_holes
import bm3d
from cv2_rolling_ball import subtract_background_rolling_ball
from skimage.morphology import remove_small_holes
from PIL import Image
# from Utils import threshold_feng

class Preprocessing:
    """
    Image Preprocessing Module for enhancing quality of document images
    -------------------------------------------------------------------
    
    1. feng_thresholding(image) -> if no argument is provided, takes original image. Returns a thresholded image
    2. otsu_thresholding(image) -> if no argument is provided, takes original image. Returns a thresholded image
    3. sauvola_thresholding(image = None, window_size=25) -> if no argument is provided, takes original image. Returns a thresholded image
    4. division_normalization(kernel_size = (33,33)) -> Pass kernel_size if necessary
    5. denoise_image_medfilt2d(kernel_size = 27) -> Pass kernel_size if necessary
    6. sharpen_image(self,img = None, sigma = 2, scale = 2)
    7. gaussian_filter(sigma = 1, size = (3,3)) -> returns cv2_gaussian, skimage_gaussian
    8. median_filter(kernel_size = 3)
    9. bilateral_filter() -> returns cv2_bilateral, skimage_bilateral
    10. nlm_denoising(strength=3.0,template_window=7,search_window=21)
    11. total_variance_denoise(weight=0.1, eps=0.0002, n_iter_max=200)
    12. bm3d_denoise()
    13. shade_correction() -> returns final_image,clahe_image
    """
    def __init__(self,filename) -> None:
        self.image = io.imread(filename,as_gray=True)
        # self.image = color.rgb2gray(np.asarray(Image.open(filename))/255.0)
        self.image_as_float = img_as_float(self.image)
        self.image_cv2 = cv2.imread(filename, 0)
        
    
    def feng_thresholding(self,img = None):
        if(img is None):
            img = self.image
        return img > threshold_feng(img)

    def otsu_thresholding(self,img = None):
        if(img is None):
            img = self.image
        return img > threshold_otsu(img)

    def sauvola_thresholding(self,img = None, window_size=25):
        if(img is None):
            img = self.image
        return img > threshold_sauvola(img,window_size)

    def division_normalization(self,kernel_size = (33,33)):
        """
        1. used for uneven illumination
        2. input should be a grayscale image
        """
        # divide gray by morphology image
        smooth = cv2.GaussianBlur(self.image, kernel_size, 0)  
        division = cv2.divide(self.image, smooth, scale=255)
        return division

    def denoise_image_medfilt2d(self, kernel_size = 27):
        """
        Input Must be Grayscale
        """
        # estimate 'background' color by a median filter
        bg = signal.medfilt2d(self.image, kernel_size)
        # save('background.png', bg)
        # compute 'foreground' mask as anything that is significantly darker than
        # the background
        mask = self.image < bg - 0.1
        # save('foreground_mask.png', mask)
        # return the input value for all pixels in the mask or pure white otherwise
        return np.where(mask, self.image, 1.0)

    """
    Unsharp mask enhances edges by subtracting an unsharp (smoothed) version of the image from the original.
    Effectively making the filter a high pass filter.
    """
    def sharpen_image(self,img = None, sigma = 2, scale = 2):
        if(img is None):
            img = self.image_as_float
        gaussian_img = gaussian(img, sigma=sigma, mode='constant', cval=0.0)
        img2 = (img - gaussian_img)*scale
        enhanced_image = img + img2  # enhanced_image = original + amount * (original - blurred)
        return enhanced_image

    def gaussian_filter(self,sigma = 1, size = (3,3)):
        img = self.image_as_float
        gaussian_using_cv2 = cv2.GaussianBlur(img, size, 0, borderType=cv2.BORDER_CONSTANT)
        gaussian_using_skimage = gaussian(img, sigma = sigma, mode='constant', cval=0.0)
        return gaussian_using_cv2,gaussian_using_skimage

    def median_filter(self,kernel_size = 3):
        median_using_cv2 = cv2.medianBlur(self.image_cv2, kernel_size)
        return median_using_cv2
    
    def bilateral_filter(self):
        bilateral_using_cv2 = cv2.bilateralFilter(self.image_cv2, 5, 20, 100, borderType=cv2.BORDER_CONSTANT)
        bilateral_using_skimage = denoise_bilateral(self.image, sigma_color=0.05, sigma_spatial=15, multichannel=False)
        return bilateral_using_cv2,bilateral_using_skimage

    def nlm_denoising(self,strength=3.0,template_window=7,search_window=21):
        dst = cv2.fastNlMeansDenoising(self.image_cv2, strength, template_window, search_window)
        return dst
    
    def total_variance_denoise(self,weight=0.1, eps=0.0002, n_iter_max=200):
        denoise_img = denoise_tv_chambolle(self.image_as_float, weight=weight, eps=eps, n_iter_max=n_iter_max, multichannel=False)
        return denoise_img

    def bm3d_denoise(self):
        """
        sigma_psd - noise standard deviation
        stage_arg: Determines whether to perform hard-thresholding or Wiener filtering.
        stage_arg = BM3DStages.HARD_THRESHOLDING or BM3DStages.ALL_STAGES (slow but powerful)
        All stages performs both hard thresholding and Wiener filtering. 
        """
        BM3D_denoised_image = bm3d.bm3d(self.image_as_float, sigma_psd=0.2, stage_arg=bm3d.BM3DStages.HARD_THRESHOLDING)
        return BM3D_denoised_image

    def shade_correction(self):
        radius=20
        final_img, background = subtract_background_rolling_ball(self.image_cv2, radius, light_background=True, use_paraboloid=False, do_presmooth=True)
        #optionally perform CLAHE to equalize histogram for better segmentation
        #otherwise the image may appear washedout. 
        clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8,8))
        clahe_img = clahe.apply(final_img)
        return final_img,clahe_img

In [ ]:
help(Preprocessing)

Help on class Preprocessing in module __main__:

class Preprocessing(builtins.object)
 |  Preprocessing(filename) -> None
 |  
 |  Image Preprocessing Module for enhancing quality of document images
 |  -------------------------------------------------------------------
 |  
 |  1. feng_thresholding(image) -> if no argument is provided, takes original image. Returns a thresholded image
 |  2. otsu_thresholding(image) -> if no argument is provided, takes original image. Returns a thresholded image
 |  3. sauvola_thresholding(image = None, window_size=25) -> if no argument is provided, takes original image. Returns a thresholded image
 |  4. division_normalization(kernel_size = (33,33)) -> Pass kernel_size if necessary
 |  5. denoise_image_medfilt2d(kernel_size = 27) -> Pass kernel_size if necessary
 |  6. sharpen_image(self,img = None, sigma = 2, scale = 2)
 |  7. gaussian_filter(sigma = 1, size = (3,3)) -> returns cv2_gaussian, skimage_gaussian
 |  8. median_filter(kernel_size = 3)
 | 

In [ ]:
#@title Load Image
# filename = "/content/drive/MyDrive/filter test images/creases2.jpg" #@param {type:'string'}
filename = "/content/drive/MyDrive/filter test images/creases4.jpeg" #@param {type:'string'}
Image = Preprocessing(filename)
img = io.imread(filename)
#@markdown NB: Change thresholding used for final based on best output observed. Default is otsu


In [ ]:
#@title Thresholding without noise removal
#@markdown Double click on output image to zoom in

gray = io.imread(filename,as_gray=True)
otsu = Image.otsu_thresholding(gray)
feng = Image.feng_thresholding(gray)
sauvola = Image.sauvola_thresholding(gray)
plot_images(img,otsu,feng,sauvola,titles = ["original","otsu",'feng',"sauvola"])

In [ ]:
#@title Median filter 2D
#@markdown Double click on output image to zoom in

denoised = Image.denoise_image_medfilt2d()
otsu = Image.otsu_thresholding(denoised)
feng = Image.feng_thresholding(denoised)
sauvola = Image.sauvola_thresholding(denoised)
final = remove_small_holes(otsu,15)
plot_images(img,denoised,final,titles=['original','denoised','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title Division Normalization
#@markdown Double click on output image to zoom in

divn = Image.division_normalization()
otsu = Image.otsu_thresholding(divn)
feng = Image.feng_thresholding(divn)
sauvola = Image.sauvola_thresholding(divn)
final = remove_small_holes(otsu,15)
plot_images(img,divn,final,titles=['original','Division Normalization','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title Sharpen Image
#@markdown Double click on output image to zoom in
sharp = Image.sharpen_image()
otsu = Image.otsu_thresholding(sharp)
feng = Image.feng_thresholding(sharp)
sauvola = Image.sauvola_thresholding(sharp)
final = remove_small_holes(sauvola,15)
plot_images(img,sharp,final,titles=['original','Sharpen','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title Gaussian FIlter
#@markdown Double click on output image to zoom in
gauss1,gauss2 = Image.gaussian_filter()
otsu1 = Image.otsu_thresholding(gauss1)
otsu2 = Image.otsu_thresholding(gauss2)
feng1 = Image.feng_thresholding(gauss1)
feng2 = Image.feng_thresholding(gauss2)
sauvola1 = Image.sauvola_thresholding(gauss1)
sauvola2 = Image.sauvola_thresholding(gauss2)
#final = remove_small_holes(otsu,15)
plot_images(img,gauss1,gauss2,titles=['original','Gaussian CV','Gaussian Ski'])
plot_images(otsu1,feng1,sauvola1,titles = ["otsu 1",'feng 1',"sauvola 1"])
plot_images(otsu2,feng2,sauvola2,titles = ["otsu 2",'feng 2',"sauvola 2"])

In [ ]:
#@markdown Double click on output image to zoom in
#@title Median filter
medianf = Image.median_filter()
otsu = Image.otsu_thresholding(medianf)
feng = Image.feng_thresholding(medianf)
sauvola = Image.sauvola_thresholding(medianf)
final = remove_small_holes(sauvola,15)
plot_images(img,medianf,final,titles=['original','Median Filter','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@markdown Double click on output image to zoom in
#@title Bilateral FIlter
bilat1,bilat2 = Image.bilateral_filter()
otsu1 = Image.otsu_thresholding(bilat1)
otsu2 = Image.otsu_thresholding(bilat2)
feng1 = Image.feng_thresholding(bilat1)
feng2 = Image.feng_thresholding(bilat2)
sauvola1 = Image.sauvola_thresholding(bilat1)
sauvola2 = Image.sauvola_thresholding(bilat2)
final = remove_small_holes(sauvola2,15)
plot_images(img,bilat1,bilat2,final,titles=['original','Bilateral CV','Bilateral Ski', 'Final'])
plot_images(otsu1,feng1,sauvola1,titles = ["otsu 1",'feng 1',"sauvola 1"])
plot_images(otsu2,feng2,sauvola2,titles = ["otsu 2",'feng 2',"sauvola 2"])

In [ ]:
#@title NLM Denoising
#@markdown Double click on output image to zoom in
nlm = Image.nlm_denoising()
otsu = Image.otsu_thresholding(nlm)
feng = Image.feng_thresholding(nlm)
sauvola = Image.sauvola_thresholding(nlm)
final = remove_small_holes(sauvola,15)
plot_images(img,nlm,final,titles=['original','NLM','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title Total Variance Denoise
#@markdown Double click on output image to zoom in
tvd = Image.total_variance_denoise()
otsu = Image.otsu_thresholding(tvd)
feng = Image.feng_thresholding(tvd)
sauvola = Image.sauvola_thresholding(tvd)
final = remove_small_holes(feng,15)
plot_images(img,tvd,final,titles=['original','TVD','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title BM3D Denoise
#@markdown Double click on output image to zoom in
bm3d = Image.bm3d_denoise()
otsu = Image.otsu_thresholding(bm3d)
feng = Image.feng_thresholding(bm3d)
sauvola = Image.sauvola_thresholding(bm3d)
final = remove_small_holes(otsu,15)
plot_images(img,bm3d,final,titles=['original','BM3D','final'])
plot_images(otsu,feng,sauvola,titles = ["otsu",'feng',"sauvola"])

In [ ]:
#@title Shade Correction
#@markdown Double click on output image to zoom in

out1,out2 = Image.shade_correction()
otsu1 = Image.otsu_thresholding(out1)
otsu2 = Image.otsu_thresholding(out2)
feng1 = Image.feng_thresholding(out1)
feng2 = Image.feng_thresholding(out2)
sauvola1 = Image.sauvola_thresholding(out1)
sauvola2 = Image.sauvola_thresholding(out2)
#final = remove_small_holes(otsu,15)
plot_images(img,out1,out2,titles=['original','Shade1','Clahe'])
plot_images(otsu1,feng1,sauvola1,titles = ["otsu 1",'feng 1',"sauvola 1"])
plot_images(otsu2,feng2,sauvola2,titles = ["otsu 2",'feng 2',"sauvola 2"])

In [ ]:
#@title BM3D backup

from skimage import io, img_as_float
import bm3d
import cv2

noisy_img = img_as_float(io.imread(filename, as_gray=True))

bm3d = bm3d.bm3d(noisy_img, sigma_psd=0.2, stage_arg=bm3d.BM3DStages.HARD_THRESHOLDING)

"""
bm3d library is not well documented yet, but looking into source code....
sigma_psd - noise standard deviation
stage_arg: Determines whether to perform hard-thresholding or Wiener filtering.
stage_arg = BM3DStages.HARD_THRESHOLDING or BM3DStages.ALL_STAGES (slow but powerful)
All stages performs both hard thresholding and Wiener filtering. 
"""

otsu = Image.otsu_thresholding(bm3d)
feng = Image.feng_thresholding(bm3d)
final = remove_small_holes(otsu,15)
plot_images(bm3d,otsu,feng,final,titles = ["BM3D","otsu",'feng',"final"])


In [ ]:
#@title test 
# import os
# from pathlib import Path
# dir = "/content/drive/MyDrive/filter test images"
# for i in os.listdir(dir):
#   print(i)

# out = "/content/drive/MyDrive/output/"
# image_dir = Path(dir)
# folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
# categories = [fo.name for fo in folders]
# n = 1
# for i, direc in enumerate(folders):
#         for file in direc.iterdir():
#           Im = Preprocessing(str(file))
#           feng = Im.feng_thresholding() 
#           otsu = Im.otsu_thresholding() 
#           sauvola = Im.sauvola_thresholding() 
#           norm = Im.division_normalization() 
#           denoise = Im.denoise_image_medfilt2d() 
#           sharp = Im.sharpen_image()
#           gausscv2,gausssk = Im.gaussian_filter() 
#           med = Im.median_filter()
#           bilcv2,bilsk = Im.bilateral_filter() 
#           nlm = Im.nlm_denoising()
#           var = Im.total_variance_denoise()
#           bm3d = Im.bm3d_denoise()
#           final,clahe = Im.shade_correction()
#           norm_feng = Im.feng_thresholding(norm) 
#           norm_otsu = Im.otsu_thresholding(norm)
#           norm_sauvola = Im.sauvola_thresholding(norm) 
#           denoise_feng = Im.feng_thresholding(denoise) 
#           denoise_otsu = Im.otsu_thresholding(denoise)
#           denoise_sauvola = Im.sauvola_thresholding(denoise)
#           sharp_feng = Im.feng_thresholding(sharp) 
#           sharp_otsu = Im.otsu_thresholding(sharp)
#           sharp_sauvola = Im.sauvola_thresholding(sharp)
#           gausscv2_feng = Im.feng_thresholding(gausscv2) 
#           gausscv2_otsu = Im.otsu_thresholding(gausscv2)
#           gausscv2_sauvola = Im.sauvola_thresholding(gausscv2)
#           gausssk_feng = Im.feng_thresholding(gausssk) 
#           gausssk_otsu = Im.otsu_thresholding(gausssk)
#           gausssk_sauvola = Im.sauvola_thresholding(gausssk)
#           med_feng = Im.feng_thresholding(med) 
#           med_otsu = Im.otsu_thresholding(med)
#           med_sauvola = Im.sauvola_thresholding(med)
#           bilcv2_feng = Im.feng_thresholding(bilcv2) 
#           bilcv2_otsu = Im.otsu_thresholding(bilcv2)
#           bilcv2_sauvola = Im.sauvola_thresholding(bilcv2)
#           bilsk_feng = Im.feng_thresholding(bilsk) 
#           bilsk_otsu = Im.otsu_thresholding(bilsk)
#           bilsk_sauvola = Im.sauvola_thresholding(bilsk)
#           nlm_feng = Im.feng_thresholding(nlm) 
#           nlm_otsu = Im.otsu_thresholding(nlm)
#           nlm_sauvola = Im.sauvola_thresholding(nlm)
#           var_feng = Im.feng_thresholding(var) 
#           var_otsu = Im.otsu_thresholding(var)
#           var_sauvola = Im.sauvola_thresholding(var)
#           bm3d_feng = Im.feng_thresholding(bm3d) 
#           bm3d_otsu = Im.otsu_thresholding(bm3d)
#           bm3d_sauvola = Im.sauvola_thresholding(bm3d)
#           final_feng = Im.feng_thresholding(final) 
#           final_otsu = Im.otsu_thresholding(final)
#           final_sauvola = Im.sauvola_thresholding(final)
#           clahe_feng = Im.feng_thresholding(clahe) 
#           clahe_otsu = Im.otsu_thresholding(clahe)
#           clahe_sauvola = Im.sauvola_thresholding(clahe)
#           save(str(n)+"/otsu.jpg",otsu)
#           save(str(n)+"/feng.jpg",feng)
#           save(str(n)+"/sauvola.jpg",sauvola)
#           save(str(n)+"/1norm.jpg",norm)
#           save(str(n)+"/1norm_otsu.jpg",norm_otsu)
#           save(str(n)+"/1norm_feng.jpg",norm_feng)
#           save(str(n)+"/1norm_sauvola.jpg",norm_sauvola)
#           save(str(n)+"/2denoise.jpg",denoise)
#           save(str(n)+"/2denoise_otsu.jpg",denoise_otsu)
#           save(str(n)+"/2denoise_feng.jpg",denoise_feng)
#           save(str(n)+"/2denoise_sauvola.jpg",denoise_sauvola)
#           save(str(n)+"/3sharp.jpg",sharp)
#           save(str(n)+"/3sharp_otsu.jpg",sharp_otsu)
#           save(str(n)+"/3sharp_feng.jpg",sharp_feng)
#           save(str(n)+"/3sharp_sauvola.jpg",sharp_sauvola)
#           save(str(n)+"/4final.jpg",final)
#           save(str(n)+"/4final_otsu.jpg",final_otsu)
#           save(str(n)+"/4final_feng.jpg",final_feng)
#           save(str(n)+"/4final_sauvola.jpg",final_sauvola)
#           save(str(n)+"/5clahe.jpg",clahe)
#           save(str(n)+"/5clahe_otsu.jpg",clahe_otsu)
#           save(str(n)+"/5clahe_feng.jpg",clahe_feng)
#           save(str(n)+"/5clahe_sauvola.jpg",clahe_sauvola)
#           save(str(n)+"/6bm3d.jpg",bm3d)
#           save(str(n)+"/6bm3d_otsu.jpg",bm3d_otsu)
#           save(str(n)+"/6bm3d_feng.jpg",bm3d_feng)
#           save(str(n)+"/6bm3d_sauvola.jpg",bm3d_sauvola)
#           save(str(n)+"/7var.jpg",var)
#           save(str(n)+"/7var_otsu.jpg",var_otsu)
#           save(str(n)+"/7var_feng.jpg",var_feng)
#           save(str(n)+"/7var_sauvola.jpg",var_sauvola)
#           save(str(n)+"/8nlm.jpg",nlm)
#           save(str(n)+"/8nlm_otsu.jpg",nlm_otsu)
#           save(str(n)+"/8nlm_feng.jpg",nlm_feng)
#           save(str(n)+"/8nlm_sauvola.jpg",nlm_sauvola)
#           save(str(n)+"/9bilcv2.jpg",bilcv2)
#           save(str(n)+"/9bilcv2_otsu.jpg",bilcv2_otsu)
#           save(str(n)+"/9bilcv2_feng.jpg",bilcv2_feng)
#           save(str(n)+"/9bilcv2_sauvola.jpg",bilcv2_sauvola)
#           save(str(n)+"/10bilsk.jpg",bilsk)
#           save(str(n)+"/10bilsk_otsu.jpg",bilsk_otsu)
#           save(str(n)+"/10bilsk_feng.jpg",bilsk_feng)
#           save(str(n)+"/10bilsk_sauvola.jpg",bilsk_sauvola)
#           save(str(n)+"/11med.jpg",med)
#           save(str(n)+"/11med_otsu.jpg",med_otsu)
#           save(str(n)+"/11med_feng.jpg",med_feng)
#           save(str(n)+"/11med_sauvola.jpg",med_sauvola)
#           save(str(n)+"/12gausscv2.jpg",gausscv2)
#           save(str(n)+"/12gausscv2_otsu.jpg",gausscv2_otsu)
#           save(str(n)+"/12gausscv2_feng.jpg",gausscv2_feng)
#           save(str(n)+"/12gausscv2_sauvola.jpg",gausscv2_sauvola)
#           save(str(n)+"/13gausssk.jpg",gausssk)
#           save(str(n)+"/13gausssk_otsu.jpg",gausssk_otsu)
#           save(str(n)+"/13gausssk_feng.jpg",gausssk_feng)
#           save(str(n)+"/13gausssk_sauvola.jpg",gausssk_sauvola)
#           n = n+1

In [ ]:
# x = ["final","clahe","bm3d","var","nlm","bilcv2","bilsk","med","gausscv2","gausssk"]
# for i,k in enumerate(x):
#   print(f"save(str(n)+\"/{i+4}{k}.jpg\",{k})")
#   print(f"save(str(n)+\"/{i+4}{k}_otsu.jpg\",{k}_otsu)")
#   print(f"save(str(n)+\"/{i+4}{k}_feng.jpg\",{k}_feng)")
#   print(f"save(str(n)+\"/{i+4}{k}_sauvola.jpg\",{k}_sauvola)")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
dir = "/content/drive/MyDrive/filter test images/blur/" #@param {type:'string'}

In [ ]:
#@title Pure Otsu
filt = "Pure Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  out = Im.otsu_thresholding()
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Pure Feng
filt = "Pure Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  out = Im.feng_thresholding()
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Pure Sauvola
filt = "Pure Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  out = Im.sauvola_thresholding()
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter2D + Otsu
filt = "Median Filter2D + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.denoise_image_medfilt2d()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter2D + Feng
filt = "Median Filter2D + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.denoise_image_medfilt2d()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter2D + Sauvola
filt = "Median Filter2D + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.denoise_image_medfilt2d()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Division Normalisation + Otsu
filt = "Division Normalisation + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.division_normalization()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Division Normalisation + Feng
filt = "Division Normalisation + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.division_normalization()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Division Normalisation + Sauvola
filt = "Division Normalisation + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.division_normalization()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Sharpen + Otsu
filt = "Sharpen + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.sharpen_image()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Sharpen + Feng
filt = "Sharpen + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.sharpen_image()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Sharpen + Sauvola
filt = "Sharpen + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.sharpen_image()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Gaussian + Otsu
filt = "Gaussian + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  op1,op2 = Im.gaussian_filter()
  out = Im.otsu_thresholding(op1)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Gaussian + Feng
filt = "Gaussian + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  op1,op2 = Im.gaussian_filter()
  out = Im.feng_thresholding(op1)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Gaussian + Sauvola
filt = "Gaussian + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  op1,op2 = Im.gaussian_filter()
  out = Im.sauvola_thresholding(op1)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter + Otsu
filt = "Median Filter + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.median_filter()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter + Feng
filt = "Median Filter + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.median_filter()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Median Filter + Sauvola
filt = "Median Filter + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.median_filter()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Bilateral Filter
#@markdown NB: For each image Otsu Feng and Sauvola are displayed unlike previous cells  
for i in sorted(os.listdir(dir)):
  print(i)
  orig = io.imread(dir+str(i))
  Im = Preprocessing(dir+str(i))
  op1,op2 = Im.bilateral_filter()
  filt = "+ Otsu"
  out1 = Im.otsu_thresholding(op1)
  out2 = Im.otsu_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Bilat1"+filt,"Bilat2"+filt])
  filt = "+ Feng"
  out1 = Im.feng_thresholding(op1)
  out2 = Im.feng_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Bilat1"+filt,"Bilat2"+filt])
  filt = "+ Sauvola"
  out1 = Im.sauvola_thresholding(op1)
  out2 = Im.sauvola_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Bilat1"+filt,"Bilat2"+filt])

In [ ]:
#@title NLM + Otsu
filt = "NLM + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.nlm_denoising()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title NLM + Feng
filt = "NLM + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.nlm_denoising()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title NLM + Sauvola
filt = "NLM + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.nlm_denoising()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title TVD + Otsu
filt = "TVD + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.total_variance_denoise()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title TVD + Feng
filt = "TVD + FEng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.total_variance_denoise()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title TVD + Sauvola
filt = "TVD + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.total_variance_denoise()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title BM3D + Otsu
filt = "BM3D + Otsu"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.bm3d_denoise()
  out = Im.otsu_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title BM3D + Feng
filt = "BM3D + Feng"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.bm3d_denoise()
  out = Im.feng_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title BM3D + Sauvola
filt = "BM3D + Sauvola"
for i in sorted(os.listdir(dir)):
  print(i)
  Im = Preprocessing(dir+str(i))
  denoised = Im.bm3d_denoise()
  out = Im.sauvola_thresholding(denoised)
  orig = io.imread(dir+str(i))
  plot_images(orig,out,titles=[str(i),filt])

In [ ]:
#@title Shade Correction
#@markdown NB: For each image Otsu Feng and Sauvola are displayed unlike previous cells  
for i in sorted(os.listdir(dir)):
  print(i)
  orig = io.imread(dir+str(i))
  Im = Preprocessing(dir+str(i))
  op1,op2 = Im.shade_correction()
  filt = "+ Otsu"
  out1 = Im.otsu_thresholding(op1)
  out2 = Im.otsu_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Shade1"+filt,"Clahe"+filt])
  filt = "+ Feng"
  out1 = Im.feng_thresholding(op1)
  out2 = Im.feng_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Shade1"+filt,"Clahe"+filt])
  filt = "+ Sauvola"
  out1 = Im.sauvola_thresholding(op1)
  out2 = Im.sauvola_thresholding(op2)
  plot_images(orig,out1,out2,titles=[str(i),"Shade1"+filt,"Clahe"+filt])